<a href="https://colab.research.google.com/github/JTassinari10/456FinalCNN/blob/main/DylanWatsonCNNFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download prasunroy/natural-images

 96% 330M/342M [00:01<00:00, 191MB/s]
100% 342M/342M [00:01<00:00, 202MB/s]


In [ ]:
! unzip /content/natural-images.zip

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
np.random.seed(25)
import random

# Load a dataset

In [ ]:
labels = ['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']
x_data = []
y_data = []
import cv2
import os
for label in labels:
  path = '/content/natural_images/{0}/'.format(label)
  folder_data = os.listdir(path)
  for image_cat in folder_data:
    image = cv2.imread(path+image_cat)

    image_changed = cv2.resize(image, (180, 180))

    x_data.append(np.array(image_changed))
    y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)

(6899,)


In [ ]:
from sklearn.preprocessing import LabelEncoder
y_data = LabelEncoder().fit_transform(y_data)

In [ ]:
#Shuffle data so it is no longer sorted
z_data = []
for i in range(len(x_data)):
  z_data.append((x_data[i], y_data[i]))


random.shuffle(z_data)

x_data = np.empty((len(x_data), 180, 180, 3), dtype=np.float32)
for i in range(len(z_data)):
  x_data[i] = z_data[i][0]

for i in range(len(z_data)):
  y_data[i] = z_data[i][1]


Split Data into Training, Validation, and Test Data

In [ ]:
#5:4:1 Split
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(x_data, y_data, test_size = 0.2)
X_train = X_train_full[:3450]
X_valid = X_train_full[3450:6209]
y_train = y_train_full[:3450]
y_valid = y_train_full[3450:6209]

# Build a neural network model

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.RandomTranslation((-0.15, 0.15), (-0.15, 0.2), input_shape=[180, 180, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding = "same", 
                               input_shape=[180, 180, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=320, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.Conv2D(filters=320, kernel_size=3, activation="relu", padding = "same"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))
tf.keras.layers.Dropout(0.25)
cnn.add(tf.keras.layers.Dense(units=64, activation="relu"))
tf.keras.layers.Dropout(0.25)
cnn.add(tf.keras.layers.Dense(units=8, activation= "softmax"))

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_translation (RandomT  (None, 180, 180, 3)      0         
 ranslation)                                                     
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 90, 90, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 128)      0

#compiling the model:


In [ ]:
cnn.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# training neural network model:


In [ ]:
history = cnn.fit(X_train, y_train, batch_size = 100, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
35/35 [==============================] - 38s 773ms/step - loss: 7.1975 - accuracy: 0.2064 - val_loss: 1.7703 - val_accuracy: 0.3209
Epoch 2/20
35/35 [==============================] - 19s 556ms/step - loss: 1.7249 - accuracy: 0.3733 - val_loss: 1.4253 - val_accuracy: 0.5379
Epoch 3/20
35/35 [==============================] - 19s 559ms/step - loss: 1.1569 - accuracy: 0.5817 - val_loss: 1.2449 - val_accuracy: 0.7023
Epoch 4/20
35/35 [==============================] - 20s 561ms/step - loss: 0.9714 - accuracy: 0.6539 - val_loss: 0.6390 - val_accuracy: 0.7733
Epoch 5/20
35/35 [==============================] - 20s 564ms/step - loss: 0.7608 - accuracy: 0.7278 - val_loss: 0.5421 - val_accuracy: 0.8163
Epoch 6/20
35/35 [==============================] - 20s 567ms/step - loss: 0.6566 - accuracy: 0.7652 - val_loss: 0.5170 - val_accuracy: 0.8144
Epoch 7/20
35/35 [==============================] - 20s 571ms/step - loss: 0.5704 - accuracy: 0.7904 - val_loss: 0.4151 - val_accuracy: 0.8453

#Evaluation



In [ ]:
score = cnn.evaluate(X_test, y_test)

44/44 [==============================] - 4s 66ms/step - loss: 0.2587 - accuracy: 0.9072


In [ ]:
print('Total loss on Testing Set:', score[0])

Total loss on Testing Set: 0.3226054012775421


In [ ]:
print('Accuracy of Testing Set:', score[1])

Accuracy of Testing Set: 0.9057971239089966
